# LangChain QA Panel App

This notebook shows how to make this app:

In [1]:
#!pip install langchain openai chromadb tiktoken pypdf panel


In [2]:
import os 
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import re
import panel as pn
import tempfile


In [3]:
pn.extension('texteditor', template="bootstrap", sizing_mode='stretch_width')
pn.state.template.param.update(
    title='',
    main_max_width="690px",
    header_background="#1e81b0",
    logo="jpmorgan-chase-co-logo.png"
)

In [4]:
custom_style = {
    'background': '#D3D3D3',
    'border': '1px solid black',
    'padding': '10px',
    'box-shadow': '5px 5px 5px #bcbcbc'
}

In [5]:
file_input = pn.widgets.FileInput(width=300)

# openaikey = pn.widgets.PasswordInput(
#     value="", placeholder="Enter your OpenAI API Key here...", width=300
# )
prompt = pn.widgets.TextEditor(
    value="", placeholder="Enter your questions here...", height=50, toolbar=False
)
run_button = pn.widgets.Button(name="Send Message", button_type = 'primary')

select_k = pn.widgets.IntSlider(
    name="Number of relevant chunks", start=1, end=5, step=1, value=2
)
select_chain_type = pn.widgets.RadioButtonGroup(
    name='Chain type', 
    options=['stuff', 'map_reduce', "refine", "map_rerank"]
)
text_to_speech = pn.widgets.TextToSpeech(name="Text to Speech Widget", auto_speak=True)
widgets = pn.Row(
    pn.Column(prompt, run_button, margin=5), width=600
)

In [6]:
def qa(file, query, chain_type, k):
    # load document
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    # select which embeddings we want to use
    embeddings = OpenAIEmbeddings()
    # create the vectorestore to use as the index
    db = Chroma.from_documents(texts, embeddings)
    # expose this index in a retriever interface
    # Pass two own query search strings (eg : First Name, Last Name)
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chain to answer questions 
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(), chain_type=chain_type, retriever=retriever, return_source_documents=True)
    result = qa({"query": query})
    print(result)
    return result

In [7]:
# result = qa("example.pdf", "what is the total number of AI publications?")

In [8]:
convos = []  # store all panel objects in a list
ai_png = pn.pane.Image('Ellipse.png', width=40)
user_png = pn.pane.Image('user.png', width=40)

verify_queries = "What is the Custodian Contact Name?"\
    "What is the Custodian Contact Address?"\
    "What is the Client Contact Name?"\
    "What is the Client Contact Telephone?"\
    "What is the Client Contact Email?"\
    "What is the Legal Entity Name?"\
    "What is the Legal Entity Identifier?"\
    "What is the Tax Identifier?"

def qa_result(_):
    # API Key from Account
    os.environ["OPENAI_API_KEY"] = ""
    # save pdf file to a temp file 
    if file_input.value is not None:
        file_input.save("temp.pdf")
        prompt_text = prompt.value
        if "Uploaded" in re.sub('<[^<]+?>', '', prompt_text) or "Upload" in re.sub('<[^<]+?>', '', prompt_text):
            result = qa(file="temp.pdf", query=verify_queries, chain_type=select_chain_type.value, k=select_k.value)
            query_result = verify_doc(result["result"])
            convos.extend([
                pn.Row(
                    pn.panel(user_png, width=10),
                    prompt_text,
                    width=600
                ),
                pn.Row(
                    pn.panel(ai_png, width=10),
                    pn.Column(
                        query_result)
                ),
                pn.Row(
                    pn.panel(ai_png, width=10),
                    pn.Column(
                        "<p>Please verify if the information is correct.</p>")
                )
            ])
            prompt.value = ""
            text_to_speech.value = "I have retrieved the following information from the document. Please verify if the informaion is correct."
        elif "Yes" in re.sub('<[^<]+?>', '', prompt_text) or "Everything" in re.sub('<[^<]+?>', '', prompt_text) or "Thank" in re.sub('<[^<]+?>', '', prompt_text):
            prompt_text = prompt.value
            print(re.sub('<[^<]+?>', '', prompt.value))
            result = get_answer((re.sub('<[^<]+?>', '', prompt.value)))
            print("result")
            print(result)
            convos.extend([
                pn.Row(
                    pn.panel(user_png, width=10),
                    prompt_text,
                    width=600
                ),
                pn.Row(
                    pn.panel(ai_png, width=10),
                    pn.Column(result)
                )
            ])
            #text_to_speech.voice = text_to_speech.voices[144]
            text_to_speech.value = result
            prompt.value = ""
            #return convos
            return pn.Column(*convos, margin=15, width=575, min_height=400)
        else:    
            result = qa(file="temp.pdf", query=prompt_text, chain_type=select_chain_type.value, k=select_k.value)
            convos.extend([
                pn.Row(
                    pn.panel(user_png, width=10),
                    prompt_text,
                    width=600
                ),
                pn.Row(
                    pn.panel(ai_png, width=10),
                    pn.Column(
                        result["result"])
                )
            ])
            prompt.value = ""
            #text_to_speech.voice = text_to_speech.voices[144]
            text_to_speech.value = re.sub('<[^<]+?>', '', result["result"])
            #return convos
        return pn.Column(*convos, margin=15, width=575, min_height=400)
    elif prompt.value:
        prompt_text = prompt.value
        print(re.sub('<[^<]+?>', '', prompt.value))
        result = get_answer((re.sub('<[^<]+?>', '', prompt.value)))
        print("result")
        print(result)
        convos.extend([
                pn.Row(
                    pn.panel(user_png, width=10),
                    prompt_text,
                    width=600
                ),
                pn.Row(
                    pn.panel(ai_png, width=10),
                    pn.Column(result)
                )
            ])
        #text_to_speech.voice = text_to_speech.voices[144]
        text_to_speech.value = result
        prompt.value = ""
        #return convos
        return pn.Column(*convos, margin=15, width=575, min_height=400)


In [9]:
qa_dict = {
    "Hi": "Hi, I'm Aria. How can I help you today?",
    "Hi Aria, I am existing client.I am looking to onboard a new Fund for Global Custody.": "Sure, do you have your Global custody agreement so I can pull up relevant information",
    "Yes, I have the document": "Could you please upload",
    "Everything looks good": "Great, I'll need a bit more information to complete the process. Shall we proceed?",
    "Yes": "Will you use the same account for cash positions?",
    "Yes, same account": "Is class action filing required for these funds?",
    "Yes, please": "Thank you. We are all set. Do you have any questions for me?",
    "Thank you": "If no further questions, you should be receiving a notification from us within 24 hours with your new account numbers. Have a nice day!"
}

doc_details = [
    "<p><b>Custodian Contact Name:</b> ",
    "<p><b>Custodian Contact Address:</b> ",
    "<p><b>Client Contact Name:</b> ",
    "<p><b>Client Contact Telephone:</b> ",
    "<p><b>Client Contact Email:</b> ",
    "<p><b>Legal Entity Name:</b> ",
    "<p><b>Legal Entity Identifier:</b> ",
    "<p><b>Tax Identifier:</b> "
]

def get_answer(question):
    print("inside get answer")
    print(question)
    if question in qa_dict:
        print(qa_dict)
        return qa_dict[question]
    else:
        return "I'm sorry, I don't have an answer to that question."

def answer_question(event):
    question = question_input.value
    answer = get_answer(question)
    answer_output.value = answer
    
def verify_doc(result):
    print("result->",result)
    format_result = "<p><b>I have retrieved the following information from the document.</b></p>"
    result_arr = result.split(". ")
    if len(result_arr) != 1:
        i = 0
        for ans in result_arr:
            parse_ans = ans.split("is")
            format_result += doc_details[i]
            format_result += parse_ans[1]
            format_result += "</p>"
            i+=1
    elif len(result.split("\n")) != 1:
        result_arr = result.split("\n")
        result_arr = result_arr[1:]
        i = 0
        for ans in result_arr:
            parse_ans = ans.split(":")
            format_result += doc_details[i]
            format_result += parse_ans[1]
            format_result += "</p>"
            i+=1
    else:
        format_result += result
        
    print(format_result)
    return format_result
    

In [10]:
def test(_):
    return pn.Column(*convos, margin=15, width=575, min_height=400)

In [11]:
qa_interactive = pn.panel(
    pn.bind(qa_result, run_button),
    loading_indicator=True,
)

In [12]:
custom_style1 = {
    'background': '#FFFFFF'
}
output = pn.WidgetBox('*Conversations:*', qa_interactive, width=630, height=440, styles=custom_style1, scroll=True)

In [13]:
# layout
pn.Column(
    pn.pane.Markdown("""
    ## ARIA Banking AI
    """),
    output,
    pn.Row(file_input,
          pn.Card(
            text_to_speech.controls(jslink=False), text_to_speech, title="Voice Settings",
              collapsed = True, width = 300
    )),
    widgets, styles=custom_style
).servable()

Column(sizing_mode='stretch_width', styles={'background': '#D3D3D3', ...})
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] WidgetBox(height=440, scroll=True, sizing_mode='fixed', width=630)
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] ParamFunction(function, _pane=Str, loading_indicator=True, sizing_mode='stretch_width')
    [2] Row(sizing_mode='stretch_width')
        [0] FileInput(width=300)
        [1] Card(collapsed=True, title='Voice Settings', width=300)
            [0] Tabs(sizing_mode='stretch_width')
                [0] WidgetBox(margin=(5, 10), name='Controls', sizing_mode='stretch_width')
                    [0] StaticText(sizing_mode='stretch_width', value='<b>Controls</b>')
                    [1] Checkbox(name='Disabled', sizing_mode='stretch_width')
                    [2] TextInput(description='The text that will be syn..., name='Value', sizing_mode='stretch_width')
                    [3] Select(description='The language o..., name='Lang', options=OrderedDict([('en-US', ...]), sizing_mode='stretch_width', value='en-US')
                    [4] FloatSlider(end=2.0, name='Pitch', sizing_mode='stretch_width', value=1.0)
                    [5] FloatSlider(end=10.0, name='Rate', sizing_mode='stretch_width', start=0.1, value=1.0)
                    [6] Select(description='The voice that w..., name='Voice', sizing_mode='stretch_width')
                    [7] FloatSlider(name='Volume', sizing_mode='stretch_width', value=1.0)
                    [8] Checkbox(name='Auto speak', sizing_mode='stretch_width', value=True)
                    [9] Button(name='Cancel', sizing_mode='stretch_width')
                    [10] Button(name='Pause', sizing_mode='stretch_width')
                    [11] Button(name='Resume', sizing_mode='stretch_width')
                    [12] Checkbox(disabled=True, name='Paused', sizing_mode='stretch_width')
                    [13] Checkbox(disabled=True, name='Pending', sizing_mode='stretch_width')
                    [14] Button(name='Speak', sizing_mode='stretch_width')
                    [15] Checkbox(disabled=True, name='Speaking', sizing_mode='stretch_width')
                    [16] ListInput(description='Returns a list o..., disabled=True, name='Voices', sizing_mode='stretch_width', type=<class 'list'>)
                    [17] ListInput(name=' voices', sizing_mode='stretch_width', type=<class 'list'>)
                [1] WidgetBox(margin=(5, 10), name='Layout', sizing_mode='stretch_width')
                    [0] StaticText(sizing_mode='stretch_width', value='<b>Layout</b>')
                    [1] TextInput(description='String identifier f..., disabled=True, name='Name', sizing_mode='stretch_width', value='Text to Speech Widget')
                    [2] LiteralInput(description='Whether the object should..., name='Align', sizing_mode='stretch_width', value='start')
                    [3] LiteralInput(description='Describes the proportiona..., name='Aspect ratio', sizing_mode='stretch_width')
                    [4] LiteralInput(description='Background color o..., name='Background', sizing_mode='stretch_width')
                    [5] ListInput(description='CSS classes t..., name='Css classes', sizing_mode='stretch_width', type=<class 'list'>)
                    [6] Select(description='The design system t..., name='Design', options=OrderedDict([('None', ...]), sizing_mode='stretch_width')
                    [7] IntInput(description='The height of the compone..., name='Height', sizing_mode='stretch_width', start=0)
                    [8] IntInput(description='Minimal width o..., name='Min width', sizing_mode='stretch_width', start=0)
                    [9] IntInput(description='Minimal height o..., name='Min height', sizing_mode='stretch_width', start=0)
                    [10] IntInput(description='Minimal width o..., name='Max width', sizing_mode='stretch_width', start=0)
                    [11] IntInput(description='Minimal height o..., name=